# Setup

In [1]:
# Install Diffusers Library and Requirements for DreamBooth
#!git clone -q https://github.com/ozgung/diffusers.git
#!pip install -qq ./diffusers
!pip install -qq -r ./diffusers/examples/dreambooth/requirements.txt
!pip install -qq bitsandbytes

In [2]:
import torch
torch.cuda.is_available()


True

In [3]:
from accelerate.utils import write_basic_config

write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

#Prepare Data

In [4]:
# Data Config
DS_NAME = "ozgung/red-bowl"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
INSTANCE_PROMPT = "a photo of a sks red bowl"
TEST_PROMPT = "a photo of a sks red bowl as a painting"


In [5]:
from huggingface_hub import snapshot_download
import shutil, os.path

local_dir = os.path.join('./IN', DS_NAME)
output_dir = os.path.join("./OUT", DS_NAME)

snapshot_download(
    DS_NAME,
    local_dir=local_dir,
    repo_type="dataset",
    allow_patterns=["*.jpeg", "*.jpg", "*.png"],
    ignore_patterns=[".gitattributes","*.md"],
)

shutil.rmtree(os.path.join(local_dir,'.huggingface/'))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

IMG_2874.jpeg:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

IMG_2871.jpeg:   0%|          | 0.00/1.77M [00:00<?, ?B/s]

IMG_2873.jpeg:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

...

IMG_2878.jpeg:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

IMG_2876.jpeg:   0%|          | 0.00/2.90M [00:00<?, ?B/s]

IMG_2879.jpeg:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

IMG_2880.jpeg:   0%|          | 0.00/2.02M [00:00<?, ?B/s]

IMG_2881.jpeg:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

#Train

In [ ]:
train_cmd = f"""
accelerate launch diffusers/examples/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path="{BASE_MODEL}"  \
  --instance_data_dir="{local_dir}" \
  --output_dir="{output_dir}" \
  --instance_prompt="{INSTANCE_PROMPT}" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --gradient_checkpointing \
  --use_8bit_adam
"""
# run command
print({train_cmd})
!{train_cmd}

{'\naccelerate launch diffusers/examples/dreambooth/train_dreambooth.py   --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5"    --instance_data_dir="./IN/ozgung/red-bowl"   --output_dir="./OUT/ozgung/red-bowl"   --instance_prompt="a photo of a sks red bowl"   --resolution=512   --train_batch_size=1   --gradient_accumulation_steps=1   --learning_rate=5e-6   --lr_scheduler="constant"   --lr_warmup_steps=0   --max_train_steps=400   --gradient_checkpointing   --use_8bit_adam\n'}
/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
2024-06-25 11:07:11.

# Inference

In [7]:
from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained(output_dir, torch_dtype=torch.float16, use_safetensors=True).to("cuda")
image = pipeline(TEST_PROMPT, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save(os.path.join(output_dir, "generated_image.png"))
image

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


OSError: Error no file named model_index.json found in directory ./OUT/ozgung/red-bowl.

In [ ]:
#copy to drive
from google.colab import drive
drive.mount('/content/drive')
